In [1]:
import xarray as xr

import qnt.ta as qnta
import qnt.data as qndata
import qnt.output as qnout
import qnt.stats as qnstats

data = qndata.stocks.load_ndx_data(min_date="2005-06-01")

close     = data.sel(field="close")
sma_slow  = qnta.sma(close, 200)
sma_fast  = qnta.sma(close, 20)
weights   = xr.where(sma_slow < sma_fast, 1, -1)

is_liquid = data.sel(field="is_liquid")
weights   = weights * is_liquid
weights = qnout.clean(weights, data, "stocks_nasdaq100")

stats = qnstats.calc_stat(data, weights.sel(time=slice("2006-01-01", None)))
display(stats.to_pandas().tail())

performance = stats.to_pandas()["equity"]
import qnt.graph as qngraph

qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

weights = weights.sel(time=slice("2006-01-01",None))

qnout.check(weights, data, "stocks_nasdaq100")
qnout.write(weights)

NOTICE: The environment variable DATA_BASE_URL was not specified. The default value is 'https://data-api.quantiacs.io/'
NOTICE: The environment variable CACHE_RETENTION was not specified. The default value is '7'
NOTICE: The environment variable CACHE_DIR was not specified. The default value is 'data-cache'


fetched chunk 1/6 0s
fetched chunk 2/6 0s
fetched chunk 3/6 0s
fetched chunk 4/6 0s
fetched chunk 5/6 0s
fetched chunk 6/6 0s
Data loaded 1s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2023-11-10,0.891605,0.001970,0.147090,-0.270768,-0.410165,-0.043570,-0.006409,0.089109,239.0,0.035099,102.083554
2023-11-13,0.892072,0.000523,0.147074,-0.270386,-0.410165,-0.043367,-0.006378,0.069307,239.0,0.035099,102.061377
2023-11-14,0.887888,-0.004690,0.147062,-0.273808,-0.410165,-0.045140,-0.006638,0.069307,239.0,0.035101,102.055363
2023-11-15,0.883647,-0.004776,0.147050,-0.277277,-0.410165,-0.046944,-0.006903,0.069307,239.0,0.035098,102.055363
2023-11-16,0.884202,0.000629,0.147034,-0.276822,-0.410165,-0.046701,-0.006867,0.069307,239.0,0.035093,102.445649


Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...
Period: 2006-01-01 - 2023-11-16
Sharpe Ratio = -0.04670131502952647


ERROR! The Sharpe Ratio is too low. -0.04670131502952647 < 1
Improve the strategy and make sure that the in-sample Sharpe Ratio more than 1.


...

NOTICE: The environment variable ENGINE_CORRELATION_URL was not specified. The default value is 'https://quantiacs.io/referee/submission/forCorrelation'
NOTICE: The environment variable STATAN_CORRELATION_URL was not specified. The default value is 'https://quantiacs.io/statan/correlation'
NOTICE: The environment variable PARTICIPANT_ID was not specified. The default value is '0'


WARNING! This strategy correlates with other strategies and will be rejected.
Modify the strategy to produce the different output.


The number of systems with a larger Sharpe ratio and correlation larger than 0.9: 1
The max correlation value (with systems with a larger Sharpe ratio): 0.9998177078832443
Current sharpe ratio(3y): -0.022683782551078332

Correlated examples:

Name               Coefficient    Sharpe ratio
---------------  -------------  --------------
Q20 Quick Start       0.999818      -0.0287646


NOTICE: The environment variable OUTPUT_PATH was not specified. The default value is 'fractions.nc.gz'


Write output: fractions.nc.gz


In [4]:
import xarray as xr

import qnt.backtester as qnbt
import qnt.data as qndata
import numpy as np
import logging

from sklearn.linear_model import RidgeClassifier

In [3]:


def load_data(period):
    return qndata.stocks.load_ndx_data(tail=period, assets=["NAS:AMZN"])



def predict_weights(market_data):

    def get_ml_model():
        
        model = RidgeClassifier(random_state=18)
        return model

    def get_features(data):
        def take_log(prices_pandas_):
            prices_pandas = prices_pandas_.copy(True)
            assets = prices_pandas.columns
            for asset in assets:
                prices_pandas[asset] = np.log(prices_pandas[asset])
            return prices_pandas
        price = data.sel(field="close").ffill("time").bfill("time").fillna(0)
        for_result = price.to_pandas()
        features_df = take_log(for_result)
        return features_df

    def get_target_classes(data):
        price_current = data.sel(field="close").dropna("time")
        price_future = price_current.shift(time=-1).dropna("time")

        class_positive = 1
        class_negative = 0

        target_is_price_up = xr.where(price_future > price_current, class_positive, class_negative)
        return target_is_price_up.to_pandas()

    data = market_data.copy(True)

    asset_name_all = data.coords["asset"].values
    features_all_df = get_features(data)
    target_all_df = get_target_classes(data)

    predict_weights_next_day_df = data.sel(field="close").isel(time=-1).to_pandas()

    for asset_name in asset_name_all:
        target_for_learn_df = target_all_df[asset_name]
        feature_for_learn_df = features_all_df[asset_name][:-1]

        target_for_learn_df, feature_for_learn_df = target_for_learn_df.align(feature_for_learn_df, axis=0, join="inner")

        model = get_ml_model()
        try:
            model.fit(feature_for_learn_df.values.reshape(-1, 1), target_for_learn_df)

            feature_for_predict_df = features_all_df[asset_name][-1:]

            predict = model.predict(feature_for_predict_df.values.reshape(-1, 1))
            predict_weights_next_day_df[asset_name] = predict
        except:
            logging.exception("model failed")
            return xr.zeros_like(data.isel(field=0, time=0))

    return predict_weights_next_day_df.to_xarray()

weights = qnbt.backtest(
    competition_type = "stocks_nasdaq100",
    load_data        = load_data,
    lookback_period  = 10,
    start_date       = "2005-01-01",
    strategy         = predict_weights,
    analyze          = True,
    build_plots      = True
)

Run last pass...
Load data...


| |#                                                | 621 Elapsed Time: 0:00:00


fetched chunk 1/1 0s
Data loaded 0s
Run strategy...
Load data for cleanup...


| |#                                               | 1279 Elapsed Time: 0:00:00


fetched chunk 1/1 0s
Data loaded 0s
Output cleaning...
fix uniq
Check liquidity...
Ok.
Normalization...
Output cleaning is complete.
Write result...


...


Ok. This strategy does not correlate with other strategies.
---
Align...
Calc global stats...
---
Calc stats per asset...
Build plots...
---
Output:


asset,NAS:AAL,NAS:AAPL,NAS:ABNB,NAS:ADBE,NAS:ADI,NAS:ADP,NAS:ADSK,NAS:AEP,NAS:AKAM,NAS:ALGN
time,,,,,,,,,,
2023-11-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Stats:


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2023-11-03,1.079250,0.003839,0.251673,-0.346202,-0.601915,0.016130,0.004059,1.0,1.0,0.144919,7.702454
2023-11-06,1.088126,0.008225,0.251653,-0.340824,-0.601915,0.017864,0.004495,1.0,1.0,0.144891,7.702454
2023-11-07,1.111253,0.021254,0.251674,-0.326814,-0.601915,0.022318,0.005617,1.0,1.0,0.144862,7.702454
2023-11-08,1.106348,-0.004415,0.251649,-0.329786,-0.601915,0.021377,0.005380,1.0,1.0,0.144836,7.702454
2023-11-09,1.094827,-0.010413,0.251635,-0.336765,-0.601915,0.019155,0.004820,1.0,1.0,0.144806,7.702454
2023-11-10,1.117852,0.021030,0.251654,-0.322817,-0.601915,0.023561,0.005929,1.0,1.0,0.144782,7.702454
2023-11-13,1.110255,-0.006796,0.251633,-0.327419,-0.601915,0.022112,0.005564,1.0,1.0,0.144756,7.702454
2023-11-14,1.135041,0.022325,0.251658,-0.312404,-0.601915,0.026788,0.006741,1.0,1.0,0.144726,7.702454
2023-11-15,1.114800,-0.017833,0.251666,-0.324665,-0.601915,0.022966,0.005780,1.0,1.0,0.144702,7.702454


---
